In [64]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from rdkit.Chem import AllChem
from toxifate import *

plt.rcParams.update({"figure.dpi": 400})
path = r"C:\Users\Roman\OneDrive - National University of Ireland, Galway\CellPainting project data\Harmony\Nv6-PerWell\blastsProfiles.csv"
fileName = path.rsplit('\\')[-1].rsplit('.')[0]
if 'tubes' in fileName:
    plotTitle = 'C2C12 myotubes'
else:
    plotTitle = 'C2C12 myoblasts'

prof = pd.read_csv(path)

prof = prof.dropna(axis=1)
loael = pd.read_csv("LOAEL.csv")
prof = prof.loc[prof['Count']>50]

In [65]:
profFiltered = StandardProfileFormatting(prof)
meanProf = profFiltered.groupby(by=['Compound','Concentration']).mean().drop(['level_0','Row','Column','Count'],axis=1)
meanProf = Drop_correlated_columns(meanProf, 0.9)
meanProf

Cell 33342 Radial Mean   \
Compound Concentration                            
ATOR     10                            0.789844   
         30                            0.953467   
         100                           0.694236   
         300                           0.353832   
         1000                          0.759458   
...                                         ...   
ZIDO     300                           2.632491   
         1000                          3.093507   
         3000                          5.216191   
         10000                         6.606260   
         30000                         8.848390   

                        Cell 33342 Radial Mean Ratio SER-Bright  \
Compound Concentration                                            
ATOR     10                                            0.165438   
         30                                            0.895915   
         100                                           0.289910   
         300                                           0.753050   
         1000                                          1.075386   
...                                                         ...   
ZIDO     300                                           3.997854   
         1000                                          5.493552   
         3000                                          8.400457   
         10000                                         9.951713   
         30000                                        12.939485   

                        Cell 33342 Radial Mean Ratio SER-Dark  \
Compound Concentration                                          
ATOR     10                                         -0.564572   
         30                                         -0.628126   
         100                                        -0.669564   
         300                                        -0.010298   
         1000                                        0.795371   
...                                                       ...   
ZIDO     300                                         1.321695   
         1000                                        2.475364   
         3000                                        2.289647   
         10000                                       3.185389   
         30000                                       3.524389   

                        Cell 33342 Radial Relative Deviation SER-Hole  \
Compound Concentration                                                  
ATOR     10                                                 -0.317316   
         30                                                  0.512273   
         100                                                -0.275308   
         300                                                -0.251773   
         1000                                               -1.420839   
...                                                               ...   
ZIDO     300                                                 0.826250   
         1000                                                0.319213   
         3000                                                0.590565   
         10000                                               0.054044   
         30000                                              -0.527833   

                        Cell 488 Axial Length Ratio   \
Compound Concentration                                 
ATOR     10                                -0.392057   
         30                                -0.578478   
         100                                0.205440   
         300                                0.019381   
         1000                               0.240927   
...                                              ...   
ZIDO     300                               -0.515347   
         1000                              -1.234641   
         3000                              -0.825909   
         10000                             -1.363116   
         30000      

In [66]:
from rdkit import Chem
from rdkit.Chem import AllChem
import mols2grid

# List of SMILES strings
smiles_list = ['C(CCC(=O)O)CCN', 'CC(C)C1=C(C(=O)NC2=CC=CC=C2)C(=C(N1CC[C@@H](O)C[C@@H](O)CC(O)=O)C1=CC=C(F)C=C1)C1=CC=CC=C1', 'CC(C)C1=C(C(=C(C(=N1)C(C)C)COC)C2=CC=C(C=C2)F)C=CC(CC(CC(=O)O)O)O', 'N.N.Cl[Pt]Cl', 'CC1=CN([C@H]2O[C@@H](CO)[C@H](O)[C@H]2F)C(=O)NC1=O', 'CC(C)(C(=O)O)OC1=CC=C(C=C1)Cl', 'CC(=O)NC1CCC2=CC(=C(C(=C2C3=CC=C(C(=O)C=C13)OC)OC)OC)OC', 'CCCCCCCCCC(=O)N[C@@H](CC1=CNC2=C1C=CC=C2)C(=O)N[C@H](CC(N)=O)C(=O)N[C@@H](CC(O)=O)C(=O)N[C@H]1[C@@H](C)OC(=O)[C@H](CC(=O)C2=CC=CC=C2N)NC(=O)[C@@H](NC(=O)[C@@H](CO)NC(=O)CNC(=O)[C@H](CC(O)=O)NC(=O)[C@@H](C)NC(=O)[C@H](CC(O)=O)NC(=O)[C@H](CCCN)NC(=O)CNC1=O)[C@H](C)CC(O)=O', 'CC1CC2C3CCC4=CC(=O)C=CC4(C3(C(CC2(C1(C(=O)CO)O)C)O)F)C', 'CC1C(C(CC(O1)OC2CC(CC3=C2C(=C4C(=C3O)C(=O)C5=C(C4=O)C(=CC=C5)OC)O)(C(=O)CO)O)N)O', 'CCOC(=O)CF', '[H][C@]12COC(=O)[C@]1([H])[C@H](C1=CC(OC)=C(O)C(OC)=C1)C1=CC3=C(OCO3)C=C1[C@H]2O[C@@H]1O[C@]2([H])CO[C@@H](C)O[C@@]2([H])[C@H](O)[C@H]1O', 'CCOC(=O)C=C(C)C=CC=C(C)C=CC1=C(C(=C(C=C1C)OC)C)C', '[H][C@]1(CC[C@H](O)C2=CC=C(F)C=C2)C(=O)N(C2=CC=C(F)C=C2)[C@]1([H])C1=CC=C(O)C=C1', 'OC[C@H]1O[C@H]([C@@H](F)[C@@H]1O)N1C=C(I)C(=O)NC1=O', 'CC1=CC(=C(C=C1)C)OCCCC(C)(C)C(=O)O', 'CCN(CCO)CCCC(C)NC1=C2C=CC(Cl)=CC2=NC=C1', 'CNC(=NS(=O)(=O)C1=CC=C(Cl)C=C1)N1C[C@@H](C(=N1)C1=CC=C(Cl)C=C1)C1=CC=CC=C1', 'CN1CCN(CC2=CC=C(C=C2)C(=O)NC2=CC(NC3=NC=CC(=N3)C3=CN=CC=C3)=C(C)C=C2)CC1', 'CC1=C(C=NO1)C(=O)NC2=CC=C(C=C2)C(F)(F)F', 'CC1=C(C=CC(=C1)Cl)OC(C)C(=O)O', 'COC1=NC(=NC2=C1N=CN2[C@H]3[C@H]([C@@H]([C@H](O3)CO)O)O)N', 'CN1CCN(CC1)C1=NC2=CC=CC=C2NC2=C1C=C(C)S2', 'CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=O)NOCCO', 'CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C@H](C=C2)C)CC[C@@H]3C[C@H](CC(=O)O3)O)C', 'CCN(CC)CCNC(=O)C1=C(NC(=C1C)C=C2C3=C(C=CC(=C3)F)NC2=O)C', 'CC(C)(C)C(CCC1=CC=C(C=C1)Cl)(CN2C=NC=N2)O', 'C[C@@H](C1=NC=NC=C1F)[C@](CN2C=NC=N2)(C3=C(C=C(C=C3)F)F)O', 'CN(C)C1=CC=C(C=C1)N(C)C', 'CC1=CN(C(=O)NC1=O)C2CC(C(O2)CO)N=[N+]=[N-]']

name_list = ['NCAP', 'ATOR', 'CERI', 'CISP', 'CLEV', 'CLOF', 'COLC', 'DAPT', 'DEXA', 'DOXO', 'ETHF', 'ETOP', 'ETRE', 'EZET', 'FIAL', 'GEMF', 'HYCQ', 'IBIP', 'IMAT', 'LEFL', 'MCPP', 'NELA', 'OLAN', 'SELU', 'SIMV', 'SUNI', 'TEBU', 'VORI', 'WURS', 'ZIDO']
# Convert SMILES to RDKit Mol objects
mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]

# Calculate Morgan fingerprints for each molecule
fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in mols]
mols2grid.display(mols)

MolGridWidget()

In [87]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import DataStructs

def compute_correlation(smiles_list, df):
    fps = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        fp = rdMolDescriptors.GetMorganFingerprint(mol, 2, nbits = 1024)
        fps.append(fp)
    
    fps_df = pd.DataFrame(fps)
    corr = fps_df.corrwith(df)
    return corr

simpleProf = meanProf.reset_index()
simpleProf = simpleProf.loc[simpleProf['Concentration'] == 1000]

arrFingerprint = []
for molecule in fps:
    arrFingerprint.append(np.unpackbits(np.frombuffer(DataStructs.BitVectToBinaryText(molecule), dtype=np.uint8), bitorder='little'))

dfFingerprints = pd.DataFrame(arrFingerprint)
dfFingerprints = dfFingerprints.astype(float)
dfFingerprints['Compound'] = name_list
dfFingerprints.set_index('Compound', inplace=True)
simpleProf.reset_index(inplace=True)
simpleProf.set_index('Compound', inplace=True)
simpleProf.drop(columns=['Concentration', 'index'], inplace=True)
dfFingerprints.loc[:, dfFingerprints.nunique() != 1] # drop constant columns


,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
Compound,,,,,,,,,,,,,,,,,,,,,
NCAP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ATOR,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CERI,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CISP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CLEV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CLOF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DAPT,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DEXA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
correlation = dfFingerprints.corr(min_periods=0)
correlation

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,1.000000,-0.102444,-0.034483,NaN,-0.034483,0.694808,NaN,-0.034483,-0.034483,NaN,...,-0.049629,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.102444,1.000000,-0.102444,NaN,-0.102444,0.168505,NaN,-0.102444,-0.102444,NaN,...,0.484452,-0.102444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.034483,-0.102444,1.000000,NaN,-0.034483,-0.049629,NaN,-0.034483,-0.034483,NaN,...,-0.049629,-0.034483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.034483,-0.102444,-0.034483,NaN,1.000000,-0.049629,NaN,-0.034483,-0.034483,NaN,...,-0.049629,-0.034483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
